In [6]:
import pandas as pd
import numpy as np
import math
from scipy.ndimage import gaussian_filter
from typing import Optional, Dict

from bmh.benchmark.material_deposition import MaterialDeposition, Material, Deposition
from bmh.simulation.bsl_blending_simulator import BslBlendingSimulator
from bmh.helpers.math import stdev
from bmh.helpers.stockpile_math import get_stockpile_height, get_stockpile_slice_volume

def random_walk(n, start=5, step_size=0.4):  # CHANGED: smaller step_size (smoother raw feed)
    walk = [start]
    for _ in range(n - 1):
        step = np.random.uniform(-step_size, step_size)
        next_value = max(0, walk[-1] + step)
        walk.append(next_value)
    return np.array(walk)

def weighted_avg_and_std(values, weights):
    avg = np.average(values, weights=weights)
    var = np.average((values - avg) ** 2, weights=weights)
    return avg, math.sqrt(var)

class ReclaimedMaterialEvaluator:
    def __init__(self, reclaimed: Material, x_min: float, x_max: float):
        self.reclaimed = reclaimed
        self.x_min = x_min
        self.x_max = x_max
        self._volume_stdev: Optional[float] = None

    def get_volume_stdev(self) -> float:
        if self._volume_stdev is None:
            ideal_df = self.reclaimed.data.copy()
            core_len = self.x_max - self.x_min
            ideal_height = get_stockpile_height(volume=ideal_df['volume'].sum(), core_length=core_len)
            ideal_df['x_diff'] = (ideal_df['x'] - ideal_df['x'].shift(1)).fillna(0.0)
            ideal_df['volume'] = ideal_df.apply(
                lambda row: get_stockpile_slice_volume(
                    x=row['x'],
                    core_length=core_len,
                    height=ideal_height,
                    x_min=self.x_min,
                    x_diff=row['x_diff']
                ),
                axis=1
            )
            self._volume_stdev = stdev((ideal_df['volume'] - self.reclaimed.data['volume']).values)
        return self._volume_stdev

def generate_dataset_with_f1_f2():
    BED_SIZE_X = 59
    BED_SIZE_Z = 20

    deposition_timestamps = np.linspace(0, 100, 20)
    material_timestamps = np.linspace(0, 100, 50)
    x_positions = np.random.uniform(BED_SIZE_Z * 0.5, BED_SIZE_X - BED_SIZE_Z * 0.5, 20)

    # CHANGED: stronger smoothing -> smoother feed curve (tests different "information curves")
    quality_values = random_walk(50, start=5, step_size=0.4)
    quality_values = gaussian_filter(quality_values, sigma=2.0)  # CHANGED: sigma 2.0 (was ~1.0)
    volume_values = np.ones(50) * 50

    material = Material.from_data(pd.DataFrame({
        'timestamp': material_timestamps,
        'volume': volume_values,
        'quality': quality_values
    }))

    deposition = Deposition.from_data(
        data=pd.DataFrame({
            'timestamp': deposition_timestamps,
            'x': x_positions,
            'z': [0.5 * BED_SIZE_Z] * len(x_positions)
        }),
        bed_size_x=BED_SIZE_X,
        bed_size_z=BED_SIZE_Z,
        reclaim_x_per_s=6  # unchanged on purpose
    )

    sim = BslBlendingSimulator(bed_size_x=BED_SIZE_X, bed_size_z=BED_SIZE_Z)
    reclaimed = sim.stack_reclaim(MaterialDeposition(material=material, deposition=deposition))

    y1 = weighted_avg_and_std(reclaimed.data['quality'], reclaimed.data['volume'])[1]
    evaluator = ReclaimedMaterialEvaluator(reclaimed=reclaimed, x_min=float(np.min(x_positions)), x_max=float(np.max(x_positions)))
    y2 = evaluator.get_volume_stdev()

    row = {
        'y1': y1, 'y2': y2,
        **{f'x{i+1}': q for i, q in enumerate(material.data['quality'])},
        **{f'x{i+51}': x for i, x in enumerate(deposition.data['x'])}
    }
    return pd.DataFrame([row])

# ---- driver ----
N_SAMPLES = 1000  # keep small for a test; scale up later if needed
out = pd.concat([generate_dataset_with_f1_f2() for _ in range(N_SAMPLES)], ignore_index=True)
out.to_csv('matrix_f1f2_variant_smooth.csv', index=False)
print("Saved matrix_f1f2_variant_smooth.csv")


Saved matrix_f1f2_variant_smooth.csv


Why these changes?

Lower feed volatility (step_size=0.4 vs ~1.0) and stronger smoothing (sigma=2.0 vs ~1.0) model a more homogenized upstream feed. This directly tests: “How much do different material parameter information curves affect results?” by expecting lower y1 (quality stdev after reclaim). 

Keep bed geometry, deposition count, and reclaim speed the same so the only factor changing is the feed curve (clean ablation).

In [7]:
import pandas as pd
import numpy as np
import math
from scipy.ndimage import gaussian_filter
from typing import Optional, Dict

from bmh.benchmark.material_deposition import MaterialDeposition, Material, Deposition
from bmh.simulation.bsl_blending_simulator import BslBlendingSimulator
from bmh.helpers.math import stdev
from bmh.helpers.stockpile_math import get_stockpile_height, get_stockpile_slice_volume

def random_walk(n, start=5, step_size=1.2):
    walk = [start]
    for _ in range(n - 1):
        step = np.random.uniform(-step_size, step_size)
        next_value = max(0, walk[-1] + step)
        walk.append(next_value)
    return np.array(walk)

def weighted_avg_and_std(values, weights):
    avg = np.average(values, weights=weights)
    var = np.average((values - avg) ** 2, weights=weights)
    return avg, math.sqrt(var)

class ReclaimedMaterialEvaluator:
    def __init__(self, reclaimed: Material, x_min: float, x_max: float):
        self.reclaimed = reclaimed
        self.x_min = x_min
        self.x_max = x_max
        self._volume_stdev: Optional[float] = None

    def get_volume_stdev(self) -> float:
        if self._volume_stdev is None:
            ideal_df = self.reclaimed.data.copy()
            core_len = self.x_max - self.x_min
            ideal_height = get_stockpile_height(volume=ideal_df['volume'].sum(), core_length=core_len)
            ideal_df['x_diff'] = (ideal_df['x'] - ideal_df['x'].shift(1)).fillna(0.0)
            ideal_df['volume'] = ideal_df.apply(
                lambda row: get_stockpile_slice_volume(
                    x=row['x'],
                    core_length=core_len,
                    height=ideal_height,
                    x_min=self.x_min,
                    x_diff=row['x_diff']
                ),
                axis=1
            )
            self._volume_stdev = stdev((ideal_df['volume'] - self.reclaimed.data['volume']).values)
        return self._volume_stdev

def generate_dataset_with_f1_f2():
    BED_SIZE_X = 70
    BED_SIZE_Z = 20

    deposition_timestamps = np.linspace(0, 100, 20)
    material_timestamps = np.linspace(0, 100, 50)

    x_positions = np.random.uniform(BED_SIZE_Z * 0.5, BED_SIZE_X - BED_SIZE_Z * 0.5, 20)

    quality_values = random_walk(50, start=5, step_size=1.2)
    quality_values = gaussian_filter(quality_values, sigma=0.5)
    volume_values = np.ones(50) * 50

    material = Material.from_data(pd.DataFrame({
        'timestamp': material_timestamps,
        'volume': volume_values,
        'quality': quality_values
    }))

    deposition = Deposition.from_data(
        data=pd.DataFrame({
            'timestamp': deposition_timestamps,
            'x': x_positions,
            'z': [0.5 * BED_SIZE_Z] * len(x_positions)
        }),
        bed_size_x=BED_SIZE_X,
        bed_size_z=BED_SIZE_Z,
        reclaim_x_per_s=8
    )

    sim = BslBlendingSimulator(bed_size_x=BED_SIZE_X, bed_size_z=BED_SIZE_Z)
    reclaimed = sim.stack_reclaim(MaterialDeposition(material=material, deposition=deposition))

    y1 = weighted_avg_and_std(reclaimed.data['quality'], reclaimed.data['volume'])[1]
    evaluator = ReclaimedMaterialEvaluator(
        reclaimed=reclaimed,
        x_min=float(np.min(x_positions)),
        x_max=float(np.max(x_positions))
    )
    y2 = evaluator.get_volume_stdev()

    row = {'y1': y1, 'y2': y2}
    row.update({f'x{i+1}': q for i, q in enumerate(material.data['quality'])})
    row.update({f'x{i+51}': x for i, x in enumerate(deposition.data['x'])})
    return row  # return a dict (not a DataFrame)

# ---- driver
N = 200_000
output_folder = '/Users/keithkumar/Desktop/Masters/Dissertation/ML evaluation'
combined_file_path = f'{output_folder}/matrix_f1f2_200000.csv'

rows = [generate_dataset_with_f1_f2() for _ in range(N)]
combined_data = pd.DataFrame.from_records(rows)
combined_data.to_csv(combined_file_path, index=False)
print(f"Matrix dataset saved to {combined_file_path} with {len(combined_data)} rows")


Matrix dataset saved to /Users/keithkumar/Desktop/Masters/Dissertation/ML evaluation/matrix_f1f2_200000.csv with 200000 rows


Longer bed (BED_SIZE_X=70) and faster reclaim (reclaim_x_per_s=8) probe operational parameter effects on y2 (volume stability), tying to your “optimization parameters / knee region” exploration. Faster reclaim typically amplifies volume fluctuations → higher y2 likely. 

Choppier raw feed (step_size=1.2, sigma=0.5) provides a contrasting “information curve” to Block 1, to see how the same feature layout responds when both upstream variability and operational parameters change. This should raise y1 and interact with the reclaim change.

100k vs 200k (why 200k was chosen)
